In [ ]:
import requests
from bs4 import BeautifulSoup

# セッションを開始
session = requests.Session()

# ログインページのURL
login_url = "https://syllabus.sfc.keio.ac.jp/users/sign_in?locale=ja&return_to=%2Fcourses%3Fbutton%3D%26locale%3Dja%26page%3D3%26search%255Bday_codes%255D%255B%255D%3D2%26search%255Bdepartments%255D%255B%255D%3D23%26search%255Bsemester%255D%3Dfall%26search%255Bsfc_guide_title%255D%3D%26search%255Bsub_semester%255D%3D%26search%255Bsummary%255D%3D%26search%255Bteacher_name%255D%3D%26search%255Btime_codes%255D%255B%255D%3D2%26search%255Btime_codes%255D%255B%255D%3D3%26search%255Btitle%255D%3D%26search%255Byear%255D%3D2024"

# ログインに必要なデータ (ユーザー名とパスワード)
login_payload = {
    'user[cns_account]': '',
    'user[password]': ''
}

# ログインリクエストを送信
response = session.post(login_url, data=login_payload)

# ログイン成功したかどうかを確認
if response.ok:
    print("Login successful!")
else:
    print("Login failed!")



In [44]:

# 授業リンクを格納するリスト
course_links_list = []
import csv
import time

# ページ番号を指定してスクレイピングを繰り返す
for page in range(1, 28): 
    syllabus_url = f"https://syllabus.sfc.keio.ac.jp/courses?button=&locale=ja&page={page}&search%5Bcommunication_type%5D%5B%5D=on_campus&search%5Bdepartments%5D%5B%5D=23&search%5Bobjective%5D=&search%5Bsemester%5D=fall&search%5Bsfc_guide_title%5D=&search%5Bsub_semester%5D=&search%5Bsummary%5D=&search%5Bteacher_name%5D=&search%5Btitle%5D=&search%5Byear%5D=2024"
    
    syllabus_page = session.get(syllabus_url)
    
    soup = BeautifulSoup(syllabus_page.content, 'html.parser')
    div_elements = soup.find_all('div', class_='detail-btn-wrapper')
    for div in div_elements:
        a_element = div.find('a', class_='detail-btn')
        course_url = "https://syllabus.sfc.keio.ac.jp" + a_element['href']
        course_links_list.append(course_url)
        time.sleep(1)
import pandas as pd

df = pd.DataFrame(course_links_list, columns=["Course Link"])
df.to_csv('course_links_taimen.csv', index=False, encoding='utf-8')

In [46]:
# 対面授業以外
# 授業リンクを格納するリスト
course_links_list = []
import csv
import time

# ページ番号を指定してスクレイピングを繰り返す
for page in range(1, 3):  # ページ数の範囲を適宜調整
    syllabus_url = f"https://syllabus.sfc.keio.ac.jp/courses?button=&locale=ja&page={page}&search%5Bcommunication_type%5D%5B%5D=online_live&search%5Bcommunication_type%5D%5B%5D=online_ondemand&search%5Bdepartments%5D%5B%5D=23&search%5Bobjective%5D=&search%5Bsemester%5D=fall&search%5Bsfc_guide_title%5D=&search%5Bsub_semester%5D=&search%5Bsummary%5D=&search%5Bteacher_name%5D=&search%5Btitle%5D=&search%5Byear%5D=2024"
    
    # 各ページにリクエストを送信
    syllabus_page = session.get(syllabus_url)
    
    # BeautifulSoupでHTMLをパース
    soup = BeautifulSoup(syllabus_page.content, 'html.parser')
    div_elements = soup.find_all('div', class_='detail-btn-wrapper')
    for div in div_elements:
        a_element = div.find('a', class_='detail-btn')
        course_url = "https://syllabus.sfc.keio.ac.jp" + a_element['href']
        course_links_list.append(course_url)
        
import pandas as pd

df = pd.DataFrame(course_links_list, columns=["Course Link"])
df.to_csv('course_links_taimenigai.csv', index=False, encoding='utf-8')

In [47]:
csv_1 = pd.read_csv('course_links_taimen.csv')
csv_2 = pd.read_csv('course_links_taimenigai.csv')
csv_3 = pd.concat([csv_1, csv_2])
csv_3.to_csv('course_links.csv', index=False, encoding='utf-8') 

In [67]:
url = 'https://syllabus.sfc.keio.ac.jp/courses/2024_45038?locale=ja'
course_page = session.get(url)
soup = BeautifulSoup(course_page.content, 'html.parser')
soup_html = soup.prettify()
with open('soup.html', 'w', encoding='utf-8') as file:
    file.write(soup_html)

In [172]:
#改
import requests
from bs4 import BeautifulSoup
import pandas as pd

# DataFrameの初期化 
df_course_info = pd.DataFrame(columns=[
    '学部・研究科', '登録番号', '科目ソート', '科目名', '分野', '単位', '開講年度・学期', 'K-Number', 
    '同一科目', 'アスペクト', '研究会テーマ', '曜日・時限', '授業教員名', '実施形態', '授業で使う言語', '授業で英語サポート', 
    '履修条件', 'その他推奨知識', '開講場所', '授業形態', '履修制限', 
    '受け入れ予定人数', '課題提出タイプ', '学生が利用する予定機材/ソフト等', 
    '履修上の注意・留意事項', '参考文献', '連絡先メールアドレス', 'GIGAサティフィケート対象', 
    '講義概要', '主題と目標', '提出課題・試験・成績評価の方法など', '第1回', '第2回' , '第3回', '第4回', '第5回', '第6回', '第7回', '第8回', '第9回', '第10回', '第11回', '第12回', '第13回', '第14回', '第15回',
    'その他'
])

def get_course_info_from_url(course_url):
    # URLからページを取得
    response = session.get(course_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    dynamic_fields = dict.fromkeys(df_course_info.columns, '')
    
    # 科目名の取得
    h2 = soup.find('h2')
    course_name_span = h2.find('span', class_='title')
    if course_name_span:
        course_name = course_name_span.get_text(strip=True)
        dynamic_fields['科目名'] = course_name

    # 各セクションを解析
    contents_boxes = soup.find_all('div', class_='contents-box')
    for box in contents_boxes:
        h3 = box.find('h3')
        if h3:
            section_title = h3.get_text(strip=True)
            if section_title == '授業概要':
                parse_jugyo_gaiyo(box, dynamic_fields)
            elif section_title == '詳細':
                parse_shosai(box, dynamic_fields)
            elif section_title == '授業計画':
                parse_jugyo_keikaku(box, dynamic_fields)
    
    df_course_info.loc[len(df_course_info)] = [dynamic_fields[col] for col in df_course_info.columns]

def parse_jugyo_gaiyo(box, dynamic_fields):
    # '授業概要'セクション
    # 'class-info'と'syllabus-info'を解析
    class_info = box.find('div', class_='class-info')
    if class_info:
        subject = class_info.find('div', class_='subject')
        if subject:
            dl_rows = subject.find_all('dl', class_='row')
            for dl in dl_rows:
                dts = dl.find_all('dt')
                dds = dl.find_all('dd')
                for dt, dd in zip(dts, dds):
                    dt_text = dt.get_text(strip=True)
                    dd_text = dd.get_text(strip=True)
                    # DataFrameのカラム名と一致するか確認
                    colomns = df_course_info.drop(columns=['科目名'])
                    if dt_text in colomns.columns:
                        dynamic_fields[dt_text] = dd_text
    syllabus_info = box.find('div', class_='syllabus-info')
    if syllabus_info:
        dl_rows = syllabus_info.find_all('dl', class_='row')
        for dl in dl_rows:
            dt = dl.find('dt')
            dd = dl.find('dd')
            if dt and dd:
                dt_text = dt.get_text(strip=True)
                dd_text = dd.get_text(strip=True)
                if dt_text in colomns.columns:
                    dynamic_fields[dt_text] = dd_text

def parse_shosai(box, dynamic_fields):
    # '詳細'セクション
    detail_info = box.find('dl', class_='detail-info')
    if detail_info:
        dts = detail_info.find_all('dt')
        dds = detail_info.find_all('dd')
        for dt, dd in zip(dts, dds):
            dt_text = dt.get_text(strip=True)
            dd_text = dd.get_text(strip=True)
            if dt_text in df_course_info.columns:
                dynamic_fields[dt_text] = dd_text

def parse_jugyo_keikaku(box, dynamic_fields):
    # '授業計画'セクション
    lecture_info = box.find('dl', class_='lecture-info')
    if lecture_info:
        dts = lecture_info.find_all('dt')
        dds = lecture_info.find_all('dd')
        for dt, dd in zip(dts, dds):
            # 講義タイトルの取得
            title_span = dt.find('span', class_='title')
            if title_span:
                lecture_title = title_span.get_text(strip=True)
                # '第X回'を抽出
                if lecture_title.startswith('第'):
                    lecture_number = lecture_title.split(' ')[0]
                    if lecture_number in df_course_info.columns:
                        dd_text = dd.get_text(strip=True)
                        dynamic_fields[lecture_number] = lecture_title + '\n' + dd_text
                elif lecture_title == 'その他':
                    if 'その他' in df_course_info.columns:
                        dd_text = dd.get_text(strip=True)
                        dynamic_fields['その他'] = dd_text

# 出力制限を変更
pd.set_option('display.max_colwidth', None)

# CSVファイルを読み込む
course_links = pd.read_csv('course_links.csv')['Course Link']

for course_link in course_links:  # 10件だけ取得
    get_course_info_from_url(course_link)
    
df_course_info.to_csv('course_info.csv', index=False, encoding='utf-8')


In [ ]:
import pandas as pd

# 出力制限を変更
pd.set_option('display.max_colwidth', None)

# CSVファイルを読み込む
course_links = pd.read_csv('course_links.csv')['Course Link']

# 試験的に10件の授業情報を取得
for course_link in course_links[:10]:
    get_course_info_from_url(course_link)
    
df_course_info.to_csv('course_info.csv', index=False, encoding='utf-8')
    

In [ ]:
def get_course_html_from_url(course_url):
    # URLからページを取得
    response = session.get(course_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # dt と dd を取得してペアにする
    dt_elements = soup.find_all('dt')
    dd_elements = soup.find_all('dd')

    n = 1
    for dt, dd in zip(dt_elements, dd_elements):
        dt_text = dt.get_text(strip=True)
        dd_text = dd.get_text(strip=True)
        print(f'dt{n}: {dt_text}')
        print(f'dd{n}: {dd_text}')
        n += 1

get_course_html_from_url('https://syllabus.sfc.keio.ac.jp/courses/2024_39423?locale=ja')

In [171]:
url = 'https://syllabus.sfc.keio.ac.jp/courses/2024_35765?locale=ja'

df_course_info = pd.DataFrame(columns=[
    '学部・研究科', '登録番号', '科目ソート', '科目名', '分野', '単位', '開講年度・学期', 'K-Number', 
    '同一科目', 'アスペクト', '研究会テーマ', '曜日・時限', '授業教員名', '実施形態', '授業で使う言語', '授業で英語サポート', 
    '履修条件', 'その他推奨知識', '開講場所', '授業形態', '履修制限', 
    '受け入れ予定人数', '課題提出タイプ', '学生が利用する予定機材/ソフト等', 
    '履修上の注意・留意事項', '参考文献', '連絡先メールアドレス', 'GIGAサティフィケート対象', 
    '講義概要', '主題と目標', '提出課題・試験・成績評価の方法など', '第1回', '第2回' , '第3回', '第4回', '第5回', '第6回', '第7回', '第8回', '第9回', '第10回', '第11回', '第12回', '第13回', '第14回', '第15回',
    'その他'
])

def get_course_html_from_url_test(course_url):
    # URLからページを取得
    response = session.get(course_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    h2 = soup.find('h2')
    if h2:
        course_name_span = h2.find('span', class_='title')
        if course_name_span:
            course_name = course_name_span.get_text(strip=True)
            print(f'科目名: {course_name}')

get_course_info_from_url(url)
df_course_info.to_csv('course_info_test.csv', index=False, encoding='utf-8')
    
    